In [ ]:
import os
import json
import random
import copy
import torch
import utils
from transformers import AutoTokenizer,AutoConfig, AutoModel
import openai
import re
openai.api_base=''
openai.api_key = ''    #这个用closeai的接口，不需要翻墙

In [2]:

with open('../data/carb/carb_50.txt','r') as f:     
    temp_sent=f.readlines()
#with open('../data/RE-OIE2016_50.txt','r') as f:    
#    temp_sent=f.readlines()
#with open('../data/LSOIE/LSOIE_50.txt','r') as f:     
#    temp_sent=f.readlines()
#with open('data/RE-OIE2016_50_100.txt','r') as f:    
#    temp_sent=f.readlines()
#with open('data/RE-OIE2016_100_200.txt','r') as f:    
#    temp_sent=f.readlines()

print(len(temp_sent),temp_sent[0])

50 32.7 % of all households were made up of individuals and 15.7 % had someone living alone who was 65 years of age or older .



In [ ]:
def write_result(result,file_name,sent_list):
    with open(file_name,'w') as f:
        for i in range(len(sent_list)):
            #print(i)
            for triples in result[i]:
                triple1=triples.replace('<','')
                triple2=triple1.replace('>','')
                triple2=triple2.replace('\n','')
                temp_data=triple2.split('|')
                if len(temp_data)==3:
                    f.write(sent_list[i][:-1]+'\t <arg1> '+temp_data[0]+' </arg1> <rel> '+temp_data[1]+' </rel> <arg2> '+temp_data[2]+' </arg2>\t0.1\n')

In [5]:
cot_1='Extract the triples which consist of subject, predicate and object in the sentence only from the perspective of sentence structure. \
Output the triples as <subject|predicate|object> and use ; to split the triples. '
cot_2 = 'Determine whether these triples are accurate and reasonable based on the meaning of the sentence. \
If the triple is accurate and reasonable, output the original triple. \
Otherwise, modify or delete it. Use ";" to split the triples. '
cot_3 = 'Are there any triples which not extracted based on the meaning of the sentence before. \
If there is, please output the triple as <argument1|predicate|argument2> and use ";" split them. If not, output no.'
check='In all results of triples above, check that each triplet is correct. \
Output the triples you think are correct and use ";" to split them.'

In [17]:
%%time
cot1_result,cot2_result,cot3_result,check_result=[],[],[],[]       
for index,sent in enumerate(temp_sent):
    message=[{"role": "system", "content": cot_1},{"role": "user", "content": sent}]
    temp_result1=utils.gptresult(message)
    cot1_result.append(temp_result1)
    message.append({"role":"assistant","content":';'.join(temp_result1)})
    message.append({"role": "user", "content": cot_2})
    temp_result2=utils.gptresult(message)
    cot2_result.append(temp_result2)
    message.append({"role":"assistant","content":';'.join(temp_result2)})
    message.append({"role": "user", "content": cot_3})
    temp_result3=utils.gptresult(message)
    cot3_result.append(temp_result3)
    message.append({"role":"assistant","content":';'.join(temp_result3)})
    message.append({"role": "user", "content": check})
    temp_result4=utils.gptresult(message)
    check_result.append(temp_result4)

CPU times: user 3.36 s, sys: 227 ms, total: 3.59 s
Wall time: 25min 23s


In [11]:
def write_cot3(cot3result,cot2result,sent_list,file_name):
    new_cot3_result=[]
    for i in range(len(cot3result)):
        if 'no' in cot3result[i] or 'No' in cot3result[i] or 'NO' in cot3result[i]:
            new_cot3_result.append(cot2result[i])
            continue
        else:
            new_cot3_result.append(cot2result[i]+cot3result[i])
    #这样结合cot2和cot3的result就完成了
    write_result(new_cot3_result,file_name,sent_list)

In [18]:
write_result(cot1_result,'../result/zero-shot/CaRB/cot_1.carb',temp_sent)
write_result(cot2_result,'../result/zero-shot/CaRB/cot_2.carb',temp_sent)
write_cot3(cot3_result,cot2_result,temp_sent,'../result/zero-shot/CaRB/cot_3.carb')
write_result(check_result,'../result/zero-shot/CaRB/cot_3_check.carb',temp_sent)


In [ ]:
#以上，CoT就完事了，然后我要加Self-check和各自结果的比较

In [8]:
#GPT生成的提取，函数，具体用法可以运行下面备注的代码
def extract_content(input_string):
    content_list = re.findall(r'<[^>]+>\s*([^<]+)\s*</[^>]+>', input_string)
    result = '|'.join(content_list)
    return result

'''input_string = '<arg1> a </arg1> <rel> b </rel> <arg2> c </arg2>'
output = extract_content(input_string)'''

"input_string = '<arg1> a </arg1> <rel> b </rel> <arg2> c </arg2>'\noutput = extract_content(input_string)"

In [9]:
def get_triples_from_carb(carb_data,sent_data):
    #测试通过
    result,triple,sent_result=[],'',[]
    sent_result.append(carb_data[0].split('\t')[0])               #先把第一个句子加进去，方便之后做
    for data in carb_data:
        new_data=data.split('\t')
        data_sent=new_data[0]
        if data_sent not in sent_result or data==carb_data[-1]:                     #遇到新的句子了，可以记录了结果了,或走到最后一个句子了
            result.append(triple[:-1])
            sent_result.append(data_sent)
            triple=''
        triple += '<'+extract_content(new_data[1])+'>;'
    return result

In [21]:
with open('result/zero-shot/CaRB/cot_1.carb','r') as f:
    cot1_data=f.readlines()
cot1_result=get_triples_from_carb(cot1_data,temp_sent)
cot1_check=[]       #平均一个数据1块钱，时间到好一些，大约半分钟一个？
for index,sent in enumerate(temp_sent):
    message=[{"role": "system", "content": cot_1},{"role": "user", "content": sent},
             {"role": "assistant", "content": cot1_result[index]},{"role": "user", "content": check},]
    temp_result=utils.gptresult(message)
    cot1_check.append(temp_result)
write_result(cot1_check,'../result/zero-shot/CaRB/cot_1_check.carb',temp_sent)